In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.0f}'.format

In [2]:
testing_df = pd.read_csv('https://healthdata.gov/api/views/j8mb-icvb/rows.csv?accessType=DOWNLOAD')
testing_df.head(5)

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
0,AL,Alabama,1,Region 4,Positive,2020/03/01,16,16,POINT (-86.844516 32.756889)
1,AL,Alabama,1,Region 4,Negative,2020/03/01,96,96,POINT (-86.844516 32.756889)
2,AL,Alabama,1,Region 4,Negative,2020/03/02,72,168,POINT (-86.844516 32.756889)
3,AL,Alabama,1,Region 4,Positive,2020/03/02,6,22,POINT (-86.844516 32.756889)
4,AL,Alabama,1,Region 4,Negative,2020/03/03,94,262,POINT (-86.844516 32.756889)


In [3]:
positive = testing_df[(testing_df['overall_outcome'] == 'Positive')]
positive.head()

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
0,AL,Alabama,1,Region 4,Positive,2020/03/01,16,16,POINT (-86.844516 32.756889)
3,AL,Alabama,1,Region 4,Positive,2020/03/02,6,22,POINT (-86.844516 32.756889)
5,AL,Alabama,1,Region 4,Positive,2020/03/03,9,31,POINT (-86.844516 32.756889)
6,AL,Alabama,1,Region 4,Positive,2020/03/04,2,33,POINT (-86.844516 32.756889)
9,AL,Alabama,1,Region 4,Positive,2020/03/05,6,39,POINT (-86.844516 32.756889)


In [4]:
negative = testing_df[(testing_df['overall_outcome'] == 'Negative')]
negative.head()

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
1,AL,Alabama,1,Region 4,Negative,2020/03/01,96,96,POINT (-86.844516 32.756889)
2,AL,Alabama,1,Region 4,Negative,2020/03/02,72,168,POINT (-86.844516 32.756889)
4,AL,Alabama,1,Region 4,Negative,2020/03/03,94,262,POINT (-86.844516 32.756889)
7,AL,Alabama,1,Region 4,Negative,2020/03/04,0,262,POINT (-86.844516 32.756889)
8,AL,Alabama,1,Region 4,Negative,2020/03/05,61,323,POINT (-86.844516 32.756889)


In [5]:
results_df = pd.merge(positive, negative, left_on=['state', 'state_name', 'state_fips', 'fema_region', 'date'], right_on=['state', 'state_name', 'state_fips', 'fema_region', 'date'])
results_df.head()

,state,state_name,state_fips,fema_region,overall_outcome_x,date,new_results_reported_x,total_results_reported_x,geocoded_state_x,overall_outcome_y,new_results_reported_y,total_results_reported_y,geocoded_state_y
0,AL,Alabama,1,Region 4,Positive,2020/03/01,16,16,POINT (-86.844516 32.756889),Negative,96,96,POINT (-86.844516 32.756889)
1,AL,Alabama,1,Region 4,Positive,2020/03/02,6,22,POINT (-86.844516 32.756889),Negative,72,168,POINT (-86.844516 32.756889)
2,AL,Alabama,1,Region 4,Positive,2020/03/03,9,31,POINT (-86.844516 32.756889),Negative,94,262,POINT (-86.844516 32.756889)
3,AL,Alabama,1,Region 4,Positive,2020/03/04,2,33,POINT (-86.844516 32.756889),Negative,0,262,POINT (-86.844516 32.756889)
4,AL,Alabama,1,Region 4,Positive,2020/03/05,6,39,POINT (-86.844516 32.756889),Negative,61,323,POINT (-86.844516 32.756889)


In [6]:
results2 = results_df.rename({'new_results_reported_x': 'new_positive', 'new_results_reported_y': 'new_negative', 'total_results_reported_x': 'total_positive', 'total_results_reported_y': 'total_negative'}, axis=1)
results = results2.drop(columns=['overall_outcome_x', 'overall_outcome_y', 'geocoded_state_x', 'geocoded_state_y'])
results.head()

,state,state_name,state_fips,fema_region,date,new_positive,total_positive,new_negative,total_negative
0,AL,Alabama,1,Region 4,2020/03/01,16,16,96,96
1,AL,Alabama,1,Region 4,2020/03/02,6,22,72,168
2,AL,Alabama,1,Region 4,2020/03/03,9,31,94,262
3,AL,Alabama,1,Region 4,2020/03/04,2,33,0,262
4,AL,Alabama,1,Region 4,2020/03/05,6,39,61,323


In [7]:
results['daily_tests'] = results['new_positive'] + results['new_negative']
results['cumulative'] = results['total_positive'] + results['total_negative']
results

,state,state_name,state_fips,fema_region,date,new_positive,total_positive,new_negative,total_negative,daily_tests,cumulative
0,AL,Alabama,1,Region 4,2020/03/01,16,16,96,96,112,112
1,AL,Alabama,1,Region 4,2020/03/02,6,22,72,168,78,190
2,AL,Alabama,1,Region 4,2020/03/03,9,31,94,262,103,293
3,AL,Alabama,1,Region 4,2020/03/04,2,33,0,262,2,295
4,AL,Alabama,1,Region 4,2020/03/05,6,39,61,323,67,362
...,...,...,...,...,...,...,...,...,...,...,...
22744,WY,Wyoming,56,Region 8,2021/04/23,50,50310,1463,698596,1513,748906
22745,WY,Wyoming,56,Region 8,2021/04/24,41,50351,841,699437,882,749788
22746,WY,Wyoming,56,Region 8,2021/04/25,29,50380,592,700029,621,750409
22747,WY,Wyoming,56,Region 8,2021/04/26,46,50426,597,700626,643,751052


In [8]:
results.to_csv('covid_tests.csv', index=False)

In [9]:
cal_tests = results[results['state_name'] == 'California']
cal_tests.to_csv('ca_covid_tests.csv', index=False)